# Lab 7: PID Control

In previous lab sheets we have developed the following understanding and functionality:
- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.
- Developed a bang-bang controller, using logic to control the robot.
- Implement a calibration routine for the ground sensor.
- Developed a weighted-measurement of the line sensors, calculated an `error signal`, and used a proportional controller to steer the robot to produce line-following behaviour.
- Implemented a Finite State Machine to better organise your code, and to efficiently manage the operation of your robot.
- Completed `interrupt service routine` code to count the number of encoder pulses as the wheels rotate.
- Developed kinematics and odometry to track the robot position over time.  


In this labsheet we will investigate a PID controller.  A PID controller can be used in multiple contexts in the line following challenge:
- to control the speed of the wheel rotation, so that they become a more consistent and reliable subsystem.
- to control the heading of the robot, with respect to line following.
- to control the heading of the robot, with respect to $\theta$ in kinematics, allowing for good straight-line travel.
- to control the rotation of the robot, with respect to $\theta$ in kinematics, in order to make precise turn motions.
- to control the travel velocity of the robot, with respect to the distance from origin in kinematics, to allow the robot to return to home in a controlled manner.

Whilst this may seem like a long list, the PID approach is quite general and you should find it relatively easy to re-apply the techinque into different contexts.  This is one of the strengths of PID control.  

This labsheet has proven a little difficult for students in the past.  There is a supporting video available on Blackboard to help you make progress.
> Robotic Systems > Recordings > Recorded Videos > 3Pi Coding

To work on PID, we will first implement a measure of rotation velocity of the 3Pi+ wheels.  With a PID controller, we will then be able to tell the robot to maintain a fixed velocity for each wheel.  This has the clear advantage that:
- if the wheel meets any resistance, the PWM signal will automatically be increased to compensate and restore the velocity.
- if the robot is effected by a decreasing battery power, the PWM signal will automatically be increased to compensate and restore the velocity.
- even if the motors operate differently, setting a desired velocity for left and right motors will effectively match the motors and allow for good straight line travel.


<hr><br><br><br><br>

# Rotation Velocity Estimation

In previous labsheets you have developed code to:
- for various tasks, calculate a period of elapsed time, in milliseconds and microseconds, using `millis()` and `micros()`.
- developed code to effect encoder counts, and to use encoder counts for kinematics.

In this exercise both of these components will be used to estimate the rotation velocity of a wheel.  Recall that:
<p align="center">
$v = d / \Delta t$
</p>

Where $v$ is velocity, $d$ is distance, and $\Delta t$ is the change in time.  

On the 3Pi+ we actually have three reasonable locations to implement an estimation of rotation velocity:
1. Within `loop()`, allowing for some variation of time elapsed $\Delta t$ via `millis()` or `micros()`, and recording the difference in encoder counts as $d$.
2. Within the encoder ISRs.  Here, $d$ would always equal 1, because the ISR activates with every rotation of the wheel.  $\Delta t$ would need to be captured.
3. Within a `Timer` ISR, where the `Timer` would be configured so that $\Delta t$ could be fixed to a known period, and the change in encoder counts would be $d$.

In the following exercise, we will adopt the `loop()` method because it is quick to implement and sufficient to complete the line following challenge.  The other two are left to your curiousity.

## Exercise 1: Rotation Velocity Estimation

**Decompose the problem:** In this exercise, follow the normal development of working with just 1 wheel to begin with, and then update your code for both wheels when it is appropriate.

1. **Hypothesise:** What is a serious limitation of implementing the velocity estimate within the encoder ISR?  This is a tricky question!

2. It is recommended to start a new sketch to keep your program as simple as possible.  Include your code to operate the `encoders`.  Once you get this working and refactor your code, you might want to merge it back in with your main software.
  - Implement some `global variables` to track the change of encoder counts for each iteration of `loop()`.
  - Implement some `global variables` to track the change of time per iteration of `loop()`.
  - Implement a `global variable` to store an estimation of rotation velocity.

3. Implement code within `loop()` to:
  - Capture a change of encoder counts.  Remember to pay attention to your variable `data types`.
  - It is not strictly necessary to convert the change in encoder counts to radians or mm, this is at your discretion.
  - Capture the time elapsed between `loop()` iterations.
  - Divide the change of encoder by the time elapsed.
  - Store the result into your global rotation velocity variable.  Remember to think about a suitable `data type`.
  - **Validate:** Use Serial to view your rotation velocity estimation.  You may see a very rough looking signal when you move the wheel by hand.  
  - **Validate:** Use your motor command functions to see how the signal appears at different motor power settings.

4. Either use `delay()` or `millis()` (within an `if()` statement) to investigate how the frequency of calculation this velocity estimate changes quality of the signal.
  - **Hypothesise:** You may see a very spikey or scattered plot. What might be an underlying cause of this?

5. **Refactor:** Before you move on to the second wheel, consider updating your code to either be contained within a `function` or within a `class`.

6. Consider implementing a `low pass filter` for your velocity estimate signal.  This can be quickly and crudely achieved with the following code extract:
  - **Hypothesise:** What is an immediately obvious disadvantage to a low pass filter as provided below?

```c
// An initialised valued
float lpf = 0;
float alpha = 0.2; // value between 0:1
                   // how much weight to new readings?

// Update low pass filter
lpf = ( lpf * (1 - alpha ) ) + ( new_reading * alpha );

```


<hr><br><br><br><br>

# Getting Started with PID


In terms of our robot performance, PID is interesting because it may offer smoother control over the robot motion.  This will be achieved at two levels within the system:
1. Effective control of the wheels speeds.
2. Effective control of the heading of the robot.

In both cases, it is desirable that our robot does not have `jerk` in the motion - this is when the robot makes sudden and/or large changes in the motion.  For our small robot, rapid changes to wheel speed are likely to cause the wheels to slip on the surface.  From the last labsheet, we have been using the rotation of the wheels to estimate the robots `pose` (position and rotation).  Therefore, if the wheels slip on the surface, there will be encoder counts but the robot will not change it's `pose` in reality.  Within the line following challenge, wheel slip will severely effect the ability for your robot to complete the final objective, `return to home`.  

In the above two use cases (wheel speed and heading), we can imagine that rapid changes to heading are likely to cause a dramatic change of required wheels speeds.  This is because the wheel speed will be responsible for orientating the robot to a new heading.  Therefore, it is desirable for our heading to change in a continuous manner (not sudden), just as it is desirable for wheel speed to change in a continuous manner.  

PID will allow us to utilise a continuous control signal relative to some measurement of the system.  The measurement could come from:
- kinematics, such as theta (rotation) or a point to travel to (x,y)
- the position of the line under the robot
- the speed of rotation of the wheels

A nice example of PID being used for smooth motion control is the following video of an e-puck robot tracking a ball with a low-resolution camera: <a href="https://www.youtube.com/watch?v=Pga71leqf1A">YouTube</a>.  Whilst not documented, it appears at least two PID controllers are being used:
(1) tracking left to right position of ball, (2) tracking a set distance from the ball.




<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/PID_Overview.png?raw=true">
</p>

The above diagram illustrates the general form of a PID controller.  Outside of the blue-dotted box, we can see that a measurement from the system output, $y(t)$ is combined with a desired measurement, $r(t)$, to form the error signal, $e(t)$:

> $e(t) = r(t) - y(t)$


In a previous labsheet, you will have already implemented a `proportional controller` (P-controller) for line following, one of the three P, I and D elements inside the blue-dotted box.  The `integral` and `derivative` components were not investigated.  

The P-Controller implementation allows us to address many parts of the above diagram:
- the **`measurement`**, $y(t)$, was the output from the weighted-measurement calculation, which estimated the line position under the ground sensor in the range [ -1.0 : +1.0 ].
- the **`demand`**, $r(t)$, was set to 0, because it was desired to have the line positioned central between the ground sensors, which equated to a measurement value of 0.  Sometimes `demand` is also known as `setpoint`.
- the **`error signal`**, $e(t)$, meaning the difference between the measurement and the demand, was therefore $e(t) = 0 - y(t)$.
- the **`feedback signal`**, $u(t)$, was calculated by multiplying the error signal by a maximum motor power.  This had the effect of proportionally scaling motor power with respect to the error.
- therefore, the **`proportional gain`**, $K_{p}$, was set as the maximum motor power.  
- the `process` box above was the motor and wheel subsystem of the 3Pi+ robot.  In receiving the feedback signal (`analogWrite()`), the system process was altered, and so the measurement of the line sensors was effected.

We can make some interesting observations:
- the `demand` and the `measurement` were of the same units, because the PID controller is aiming to minimise the `error signal` to 0.  
- the `feedback signal`, $u(t)$, is of different units.  In the line following implementation, the feedback signal was an 8-bit value [0:255] sent to the Arduino function `analogWrite()` to send power to the motor.
- the PID controller is therefore trying to match $r(t)$ and $y(t)$ by manipulating the feedback signal, $u(t)$.
- the effect of the feedback signal $u(t)$ is to effect physical interaction of the robot with the environment, resulting in the new measurement $y(t)$.

In general terms, we can consider the PID controller as acting to transform the error signal into a feedback signal.  The question for a PID controller designer is:
> How can the error signal be usefully transformed into a feedback signal?

In the above diagram we also see the identities $K_{p}$, $K_{i}$ and $K_{d}$.  These are the system `gains`, numerical constants used to manipulate the error signal.  These are the internal parts of the PID algorithm that require a human designer to `tune` (calibrate), to create the useful transformation of error signal to feedback signal.  

The `proportional` component gain, $K_{p}$, is the easiest to gain to consider.  $K_{p}$ simply multiplies with error signal $e(t)$.  If this were a line on a graph, $K_{p}$ would effect the slope of the line.  Therefore, $K_{p}$ will effectively scale a given value of $e(t)$ to produce the feedback signal $u(t)$.




<hr><br><br><br><br>

## Exercise 2: Implement a P-Controller

Similar to previous exercises when developing new functionality, it is recommended you start with a new sketch to keep your program as simple as possible.  You will need to include your encoder, motor and velocity estimation functionality.

1. **Hypothesise:** When we implemented the `proportional` controller for the line following, we knew that the error signal would be in the range [ -1.0 : +1.0 ].  When we are going to measure and set a demand of rotational velocity, this could be in the range [ -inf : +inf ].  
  - What are the implications for finding an effective value of the `gain` $K_{p}$, and it's relation to `demand` and `measurement`?
  - What other techniques, outside of PID and these labsheets have you heard of that might be applicable to help with this problem?

2. Review and update the `pid.h` source file in the template provided on Github (<a href="https://github.com/paulodowd/EMATM0054_53_23-24/tree/main/3PI_CodeStub">Github Page</a>, <a href="https://github.com/paulodowd/EMATM0054_53_23-24/raw/main/3PI_CodeStub/Labsheet_X.zip"> Download Link for Zip file </a>) :
  - Implement variables within the class to store the `p_term`, and the `feedback signal`.
  - Utilise an `initialise()` function to set the class variables to 0.  It is recommended you also use `initialise()` to set the initial value of $K_{p}$ via an arugment passed in.
  - Utilise the `update()` function to implement the P-Controller update.
  - Add function arugments: `float update( float demand, float measurement )`.
  - Implement the `error signal` calculation, $e(t) = r(t) - y(t)$.
  - Implement the `p-term`, of the form $ p = K_{p} * e $
  - Return the `p-term` as the feedback signal.

3. At the top of your main program tab (`global variables`), create a new instance of your PID class for the one motor.  Remember to call the `initialise()` function within `setup()`.

4. Within `loop()`, schedule a call to your PID class `update()` function.  A good value to start with is a 20ms interval.
  - If you have followed the above steps, your `update()` function should return the feedback signal.  Capture this feedback signal within `loop()` by assigning it to a variable.
  - When you call `update()`, you will need to add the `demand` and the `measurement`.  
    - Use the velocity estimate of your wheel as the measurement.
    - Set the `demand` to an arbitrary but sensible value.
    - Set gain $K_{p}$ to 10 - this means the error signal will be scaled by a factor of 10.
  - **Validate:** Before you activate the motors, validate:
    - your velocity esimate is functioning properly.
    - your demand appears on a plot.
    - your demand is a sensible value relative to the measurement.
    - your error signal is functioning properly.
    - your feedback signal would cause your wheel to rotate in the right direction.
      - if the relationship is the wrong way around, you can either invert the form of the error signal calculation, or change the sign of $K_{p}$.
  
5. Add code to your `loop()` to pass the feedback signal from your PID class `update()` function into the command to set the power for your motor.
  - Test your robot with motor power.  It is recommended you lift your robot off the surface for the time being.  Using a cup or the reel of black tape beneath the robot is convenient.
  - **Validate:** Use the Serial Monitor or Plotter to check that the subsequent measurements of rotation velocity are closer to your demand.
    - If the `measument` has gone away from the `demand`, your feedback signal is inverted.
    - If you see a lot of oscillation, your gain $K_{p}$ is too large.
    - If you see very little movement (but perhaps, can hear the motor), your $K_{p}$ gain is too small.

6. Attempt to find a value for the gain $K_{p}$ so that for a given sensible demand of rotation velocity, your motor will be driven with a feedback signal to match the measurement to demand.
  - Start with a "too small" value of $K_{p}$ and increase it incrementally.
  - **Hypothesise:** You will likely notice that for a value of $K_{p}$, the closer your measurement gets to the demand when the robot is operating, the more oscillation you will see.  Why might this be?


<hr><br><br><br><br>

# Steady-State Error and the I-Term

In the last exercise when tuning the `gain` $K_{p}$ for a P-Controller we observed that the closer the measurement got to the demand, the more likely it was that there was undesirable oscillation.  In fact, the most stable performance you could achieve probably looked like the following on a  plot illustration:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/PID_SteadyStateError.png?raw=true">
</p>

The above illustrated peformance has to do with the calculation of the error signal:

$e(t) = r(t) - y(t)$

In a P-Controller, as the demand and measurement draw near, the error signal tends towards zero.  This would mean that the feedback signal also tends towards zero.  In the application of `closed-loop` control of the rotation velocity of the wheel, a zero feedback signal would mean the motor becomes deactivated.  

In the above illustration, the `gain` $K_{p}$ has actually be tuned to create an uneasy equilibrium between a non-zero error signal, and a non-zero feedback signal.  This has the consequence that the system can never reach the demand, and this not reliable.  When $K_{p}$ has been tuned for equillibrium like this, it is unlikely to accept a wide range of demand inputs.  As annotated in the above, the persistent gap between the demand and measurement is known as a `steady-state error`.  

This `steady-state error` will occur for P-Controllers in systems where continuous drive is required to keep the error signal minimised.  An intuitive example is a helicopter instructed to hover at an altitude of 10 meters.  The helicopter will be under the effect of gravity, pulling it back to earth.  From an initial state on the ground, our P-Controller would drive the engine, causing lift off.  However, when the helicopter attained 10m altitude, the feedback signal would be 0, and the engine would switch off.

In contrast, we can consider a system like an inkjet printer, which needs to move the print heads to specific locations along a rail.  In this system it would be desirable that as the carriage approaches the destination, the velocity tends towards zero to stop the carriage at the correct destination. After this, no power is required to maintain the position.

Rotation velocity control of the 3Pi+ wheel is more like the helicopter, because frction will quickly work to slow the wheel when power is removed.  

The `I-term` in a PID controller will work to mitigate `steady-state error`.  For the time that the system exists with non-zero error, The I-term will integrate the error signal.  This has the effect that the longer the system is in error, the more the I-term will contribute to the overall output feedback signal.  When using a PI-Controller, our feedback will be the contribution of both P and I terms:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/PID_PI.png?raw=true">
</p>

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/PID_ITerm.png?raw=true">
</p>

The above illustration attempts to show the following:
- at $t_{1}$, the P-term (green line) has successfully driven the system into `steady-state error`.  We can think of the P-term as providing this quick response to input, which would be useful for overcoming stiction or friction in the system.  During this time to $t_{1}$, the I-term (yellow line) has integrated all of the error that occured.  
- As the feedback signal (blue line) - being the sum of the P-term and I-term - minimises the error signal to 0, the P-term (green line) sharply falls toward 0 (recall, P is proportional to the error, which has become near 0).  However, without the P-term, the system will remain with error, until the I-term continues to integrate more of the error signal.
- At $t_{2}$, the I-term has integrated sufficient error to drive the system beyond the demand.  This is generally undesirable, but some overshoot is expected with the I-term.  This is because the I-term will operate with `lag` - that is, it takes time for the I-term to integrate (to change its value).  
- When the system has overshot the demand, it is important to note that the error signal will now have the opposite sign.  Therefore, the I-term will begin to integrate "down", and so lessen the drive of the system.  
- At point $t_{3}$, the integral has now undershot slightly, for the same reason as prior.  However, the integral term will quickly stabilise the system about the demand.

A common problem with the I-term is known as `wind-up error`.  This is where the I-term integrates continuously, if the system is not able to respond.  It is also a common bug in a PI or PID controller.  If a PI controller is being used for position control (such as the inkjet printer example earlier), the I-term can also integrate a tiny error once the system has stopped, and then cause a sudden jerk and then oscillation.  The I-term is most effective in a system which requires continuous drive to minimise the error signal.  In these systems, it is beneficial that the I-term maintains a non-zero value, and it will `wind-down` to match the demand when it has overshot.




## Exercise 3: Implement a PI-Controller

When implementing on a microcontroller, the integration can be approximated in the following form:

<p align="center">
$I = K_{i} * \sum_{i=1}^{k}  (e(t_{i})\Delta t)$
</p>

Where $k$ is current iteration, from activation at $i=1$.  If this equation is confusing, you could review some pseudocode implementations (such as the <a href="https://en.wikipedia.org/wiki/Proportional%E2%80%93integral%E2%80%93derivative_controller#Pseudocode">Wikipedia page</a>).  

To start these exercises, work with your P-Controller from the last exercise that has stable performance but steady-state error.  There is a high likelihood your robot will suddenly drive away.  Therefore, please begin by lifting your robot off the surface:

<p align="center">
<img  src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3PI_Lifted.png?raw=true">
</p>

1. **Validate:** Start by implementing the I-term calcuation, but do not add it's contribution to the feedback signal.  
  - To implement the I-term, you will most likely need to update your PID class to calculate the period of elapsed time, $\Delta t$.  It is recommended you utilise variables inside the class to store this information, rather than passing it in.
  - Set an arbitrary but small `gain` value for the I-term, $K_{i}$, such as `0.001`.
  - Remember to multiply the current error signal by $\Delta t$ before it is integrated.
  - Check that the I-term is working by viewing it's value on the Serial Plotter.
  - Check that the I-term appears to integrate error with the right sign (in the right "direction").
  - Without adding the I-term to the feedback signal, you should see the I-term `wind-up`, and integrate to extremely large values.

2. **Validate:** Update the calculation of your feedback signal to include the I-term:
  - **Note:** you will see extreme acceleration and oscillation if your battery power is not switched on.  This is because the I-term will be integrating error whilst the system is unable to respond.  You will also see the same irratic behaviour if your code uses a `delay()` call of significant length between your PID `update()` calls.
  - If your $K_{i}$ gain is too high, you may induce oscillations into your system.
  - If your $K_{i}$ gain is very low, it may take some time to see the effect.  

3. **Experiment:** Try adjusting your PID `update()` routine to only use the I-term, an I-Controller.  What do you observe?
  - **Note:** Attempting to use an I-Controller by tuning $K_{i}$ to a high value is likely to result in very unstable behaviour.
  - I-Controllers are not normally used.

4. **Experiment:** When you add the I-term to create a PI-Controller, you are also likely to need to re-tune your P-term `gain` $K_{p}$.  
  - If you lose good performance, follow these steps:
    - disable the I-term
    - re-tune the P-term `gain` $K_{p}$ to establish stable control with `undershoot` to demand.
    - set the I-term `gain` $K_{i}$ to a very small value.
    - enable the I-term
    - slowly increment the I-term `gain` $K_{i}$
  - **Validate:** Once you have gained stable control, it is important to test your robot on the floor of the `task environment`.  You may find your `gain` values need to be adjusted slightly.  


**Help:** In these exercises, irratic or confusing behaviour from your robot can be coming from other parts of your robotic system.  If you have not done so already, you should check the code you are using to operate your motors to ensure all of the exceptional cases (likely sources of bugs and errors) are handled.  


<hr><br><br><br><br>

# The D-Term

The D-term in a PID system is generally regarded as a `dampening` contribution to the feedback signal.  This means that the D-term will lessen the effect of sudden changes in the error signal.  

The D-term therefore looks at the rate of change of the error signal:

<p align="center">
$D = K_{d} * \frac{de(t)}{dt}$
</p>

Because a PID controller is `closed-loop` control, we can consider that changes to the error signal can come from both:
- the output of the system via the measurement $y(t)$
- changes made to the demand $r(t)$:

<p align="center">
$e(t) = r(t) - y(t)$
</p>

In our system, it is much more likely that sudden changes will occur to the demand.  Examples of changes at the measurement (output) of the system would be adding a significant mass load to the robot, which would change the relationship between the motor power feedback signal and the measured performance.

The D-term will essentially be subtractive to your feedback signal (and thus, the P-term and I-term combined), and the two can quickly interact to form unstable oscillations.  Therefore, D-term should be used to mitigate infrequent and high magnitude "shocks" to the system, such as sudden changes of demand.  

Because the D-term is generally regarded as a dampening mechanism, it is often approached to remove all variation/noise in the feedback signal.  **This is not a recommended use of the D-term**.  This is because, to make the D-term effective in removing normal and persistent small noise from your feedback signal, you would need to set a very large gain for your D-term.  Consequently, for a less frequent, but higher magnitude sudden change to input or load, this high gain value would cause an extreme D-term response, and likely cause a complete control failure.  Therefore, we should expect a relatively small gain for the D-term, and for it to be effective only in infrequent events.

To give an example of using the D-term for large shocks, we can imagine that your robot is moving at high speed forwards, and a new demand is to move at high speed backwards.  This is a likely scenary for your 3Pi+ robot:  

<p align="center">
<img  src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/PID_DTerm.png?raw=true">
</p>

In the above illustration, we can see the demand (red line) moving between a positive rotation velocity to a negative rotation velocity.  At $t_{1}$, the sudden change to positive causes a very large error - note, here the error is equal to double the positive demand value, because we are moving from a negative value to positive.  The illustration shows that from a stable state where the error signal was previously 0, the P-term (green line) will provide large feedback, causing dramatic overshoot.  Assuming this is a PI-controller, the I-term (not shown) would bring the system to demand, allowing the P-term to return to 0.  However, at $t_{2}$, the demand goes negative, and the same but inverse response is shown.

In the above illustration, the D-term would be tuned to attempt to remove the overshoot visible in the pink circle.  Under such a large change of the error signal, a small D-term `gain` $K_{d}$ would be effective.  With the same value of $K_{d}$ under the normal small variation of error signal, the D-term would be negligible.  This is fine - it is desirable that your system needs as little dampening as possible.

It is likely that you will find the D-term is the least important in your coursework, and can be considered as an opportunity for further if you have time.

## Exercise 4: Implement a PID Controller

This exercise pursues a full PID controller.  From experience, the D-term is not essential to the performance of your 3Pi+ robot.  Depending on how you write your code, large changes to the input demand can be made unlikely.  However, it is still worthwhile exploring the D-term.  On a microcontroller, this can be approximated as:

<p align="center">
$D = K_{d} * \frac{e(t_{k}) - e(t_{k-1})}{\Delta t}$
</p>

Where $k$ is the current iteration of the PID update in time, and $\Delta t$ is the change in time since the last update.

1. **Validate:** Working from your PI-controller:
  - implement the D-term and set a small value for the D-term `gain` $K_{d}$.
  - To start with, do not include the D-term in the final feedback signal calculation.
  - from within `loop()`, implement a scheduled section of code to invert the demand.  For example, switch a demand between 0.5 and -0.5 every 2 seconds.
  - Use the Serial Plotter to view the D-Term response. If it is hard to view, scale the value by a factor of 10 or 100 when reporting to the Serial Plotter.
  - Check the sign (direction) of the D-term reponse.  If the D-term produces a signal that matches the sign of the P-term, you have two choices:
    - In your final feedback signal, you would want to subtract the D-term.  Recall, the D-term should dampen, or counteract sudden changes.
    - You can set the D-term `gain` to be negative, which would effectively subtract the response from the final feedback calculation.


<hr><br><br><br><br>

## Exercise 5: Refactor, Implement on both Wheels

At this point, hopefully you have a P, PI or PID controller that you are happy with.  Take some time to **refactor** your code.  Review the following:

1. A common bug with PID controller implementation is when $\Delta t = 0$, causing a division by zero error, or causing the I-term to be ineffective.  This can happen when your PID `update()` function is called very quickly (e.g. faster than 1ms if using `millis()`.  Update your code to prevent or mitigate this error.

2. A common bug with PID controllers occur if the system is placed into a state where it cannot respond to the feedback signal.  In the Line Following Challenge `task`, this may occur at any point your robot utilises a `delay()`, or de-activates motor control.  If the PID `update()` function continues to be called, the I-term will integrate to an extremely large value.  You will then see your robot suddenly go into irratic behaviour when the motors are activated again.  This will also occur if you are using USB power for some time, and then activate the battery power.
  - to prevent this, write a function called `reset()`, which will reset the variables to calculate $\Delta t$ and the I-term variables.  It is advisable to call `reset()` if there is a transition in behaviour, or any motor deactivation.  

3. As stated, implement a PID controller for the other wheel.  If you have used the `pid.h` class, this should be as simple as creating two instantiations of the class:
  - When you have completed this, you should find that setting a demand of equal speed to left and right wheels results in pretty good straight line travel.  We will improve this further in the next exercise.

```c
// Create two instances of PID for controlling
// the wheel velocity.
PID_c vel_left_pid;
PID_c vel_right_pid;
```

<hr><br><br><br><br>

# A Heading PID Controller

At this point, you should have developed a PID controller for the velocity control of your 3Pi+ wheels.  With both wheels under `closed-loop` control of velocity, you should be able to see pretty good straight line travel.  

In this section we will investigated `nested control`.  This is when a `closed-loop` control system is itself controlled by a `closed-loop` control system. It was stated that a PID controller is desirable because it can offer continuous control to the motors, and hypothetically reduce jerk in the motion.

The same PID principle can be applied to automate the demand velocity for the left and right wheels.  A third PID controller can be used to operate a `heading` for the robot:

<p align="center">
<img  src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3PI_NestedControllers.png?raw=true">
</p>

In the above, the weighted-measurement from the line sensor has been used as the `measurement` for the `heading` controller.  This controller would have a demand of `0` to follow the line.  The output of the `heading` controller would be a single feedback signal as we are already familiar with.  To produce turning behaviour, the wheels must be driven in opposite directions.  Therefore, the one feedback signal is inverted for one of the wheel rotation velocity controllers.  The `gains` of the `heading` controller would need to be carefully tuned so that the feedback signal effects sensible `velocity demands` for the wheel controllers.

<p align="center">
<img  src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/3PI_NestedControllers2.png?raw=true">
</p>

In the above diagram, the same `architecture` of nested control is used, but the demand and measurement for the `heading` controller has been switched to information from our kinematics and odometry subsystem.  To enable smooth rotation of the robot to rotate to any direction, the `heading` controller would look to minimise the difference between a demand $\theta$, and the kinematics $\theta$.  

## Exercise 5: Implement Nested-Control

1. **Hypothesise:** The line following map has many components, or different parts of the line map create different challenges.  When and how might the I-term of the `heading` controller be of particular benefit on the Line Following Challenge map?

2. Create a third instance of your PID `class`, and name it appropriately as a heading controller.  
  - Adjust your main code so that the left and right wheel controllers will now have their demand set by the output (feedback signal) of the heading controller.
  - You can use a similar technique as used before to effect both:
    - forward bias of motion
    - a turn of motion

3. **Validate:** Decide whether you wish to test against Line Following or Kinematics:
  - Follow similar steps to validate and tune your wheel PID controllers.

3. Implement a new behaviour which utilises the `heading` controller to drive in a straight line according to the kinematics $\theta$:
  - To begin with, start with a demand of $\theta = 0$, assuming your robot activates at $\theta = 0$.  
  - Next, implement `global variable` to store a demand $\theta$ as the `heading` controller demand, and use the kinematics $\theta$ as the `heading` controller measurement.  

4. Implement a new behaviour for your robot which utilises the `heading` PID controller to turn to a specific angle in the global co-ordinate frame.
  - The `atan2()` function is your <a href="https://en.wikipedia.org/wiki/Atan2">best friend</a>.

5. **Going Home:** Implement a new behaviour for your robot which utilises the `heading` PID controller to turn to face back to the origin coordinates (where the robot started), from any position away from origin:
  - **Help:** When debugging this behaviour, it is useful to be able to start your robot kinematics with initial $X$, $Y$ and $\theta$ values, and turn back towards $X = 0$, $Y = 0$.
  - **Help:** Note that, to rotate from one angle to another, there is going to be a long-way-around, and a short-way-around.  In terms of achieving stable behaviour, it is better to always take the short-way-around.
  - **Help:** It is useful to solve the problem of calculating the difference between two angles.  Your robot will have:
    - The angle of it's location from origin.
    - The robots current angle of rotation.
  - **Help:** If you use the difference between two angles as the `heading` controller measurement, the demand to effect rotation would be 0.  Discuss this with your peers if this does not make immediate sense.



<hr><br><br><br><br>

# Congratulations!

Congratulations!  You have reached the end of the last labsheet.  

If you're reading ahead - well done, that is recommended.  However, you better get started.  If you have finished the last exercises, then you are very close to solving the Line Following Challenge.  You have all the essential components in code you need to solve the task.  However, there are a few elements that have not been directly addressed by the labsheets:

- **Help:** Traversing the Gap: This component of the task has an element of `perceptual aliasing`.  It is hard for the robot to know whether it has lost the line in an ordinary sense, or finished the complete line.  It is important to consider what information is accessible to the robot, and which of these will make the most robust behaviour.
  - In robotics, we will often face a dilemma of how much specialist domain knowledge (e.g., human knowledge) to program into a solution.  It is preferable (in the spirit of automation) if the robot can be as general and as autonomously adaptable as possible.  

- **Help:** Going Home has several features which can make it challenging to debug.  Most of all, it occurs at the end of your line following routine.  You will be able to debug your code quicker if you can start your program at a system `state` just prior to Go Home.  Time spent watching your robot operate (e.g. solving line following again and again) adds up quickly.  
  - The Go Home element has a lot of trigonometry involved, and it can be useful to draw it out.
  - The Go Home element is a lot like line following, except you will be following information derived from your kinematics.
  - Getting good performance from the Go Home element can have a lot to do with a good calibration of your kinematic model.

- **Help:** A concept not introduced in the labsheets is called a `watchdog timer`.  A watchdog timer is a mechanism which monitors the passage of time, and then creates an event if a threshold is crossed.  This is similar to the `timeout` fuctionality we used for the Line Sensors.  You can implement a watchdog timer using either `millis()` or `micros()`, or if your program has good flow control, you can use a variable to count up or down with each iteration.  

The following Block Diagram, representing how information is used in a complete solution, may be a useful further reference for integrating your components into a final solution:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0054_53_23-24/blob/main/Images/SystemBlockDiagram.png?raw=true">
</p>




<hr>
# Recommended Reading

Quantisation Error: <a href="">DSP Guide for Engineers Chapter 3.</a>